In [ ]:
from sentence_transformers import CrossEncoder

# TODO fine-tune
model = CrossEncoder('cross-encoder/stsb-distilroberta-base', max_length=512)

In [ ]:
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

# TODO fine-tune
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [34]:
# Irrelevant to the model at the moment
question_title = "Who was the last pharaoh of ancient Egypt's New Kingdom? When did he reign?"
# (token, max_grade)

tokens = [
    ("Pharaoh Ramses XI", 1),
    ("Reigned from 1107 BC to 1078 BC", 1)
    ]

student_answer = ["Reigned from 1107 BC to 1100 BC", "Pharaoh Hatshepsut"]

In [35]:
def cluster_pairs(model_answer, student_answer):
  # TODO is worth it?
  """
  Perform K-means clustering for model answer tokens student answer tokens
  instead of computing all possible combinations
  """
  num_clusters = len(model_answer)

  corpus = model_answer + student_answer
  corpus_embeddings = embedder.encode(corpus)

  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)

  clustered_sentences = [[] for i in range(num_clusters)]

  for sentence_id, cluster_id in enumerate(clustering_model.labels_):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

  return clustered_sentences

In [38]:
tokens_only = [ans for ans, _ in tokens]

ret = cluster_pairs(tokens_only, student_answer)
print(ret)

[['Reigned from 1107 BC to 1078 BC', 'Reigned from 1107 BC to 1100 BC'], ['Pharaoh Ramses XI', 'Pharaoh Hatshepsut.']]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [45]:
# TODO return grade after applying threshold
scores = model.predict(ret)
ret = [(pair, score) for pair, score in zip(ret, scores)]

In [46]:
ret

[(['Reigned from 1107 BC to 1078 BC', 'Reigned from 1107 BC to 1100 BC'],
  0.6932955),
 (['Pharaoh Ramses XI', 'Pharaoh Hatshepsut.'], 0.24468662)]